In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    precip=ds['ar_labeled']#[0::2,:,:]
    precip=precip/precip
    precip=precip.fillna(0)
    
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    ivt=ds['ivt'][0::2,:,:]
    ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
    iwv=ds['iwv'][0::2,:,:]
    ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
        
    ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
    mfc=ds['mfc'][0::2,:,:]
        
    wet_spells=xr.zeros_like(precip)
    ivt_max_array=xr.zeros_like(precip)
    iwv_max_array=xr.zeros_like(precip)
    s_max_array=xr.zeros_like(precip)
    mfc_max_array=xr.zeros_like(precip)
    for j in range(precip.time.size):
        if j%100==0:
            print(j)
        precip_sel=precip[j,:,:]
        ivt_sel=ivt[j,:,:]
        iwv_sel=iwv[j,:,:]
        s_sel=s[j,:,:]
        mfc_sel=mfc[j,:,:]
        
        if i==0 and j==0:
            ones_sum=precip_sel
            
            ivt_max=ivt_sel.values
            ivt_max=np.where(precip_sel.values==1,ivt_max,np.nan)
            iwv_max=iwv_sel.values
            iwv_max=np.where(precip_sel.values==1,iwv_max,np.nan)
            s_max=s_sel.values
            s_max=np.where(precip_sel.values==1,s_max,np.nan)
            mfc_max=mfc_sel.values
            mfc_max=np.where(precip_sel.values==1,mfc_max,np.nan)

            if 1==0:
                fig=plt.figure(figsize=(18,12))
                ax=plt.subplot(1,1,1)
                test1=ones_sum.where(ones_sum>0)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,test1.values,levels=np.arange(0,11,1),cmap=plt.cm.Blues)
                plt.show()
                plt.close()

        else:
            #check where wet spell has ended
            wet_end=xr.ones_like(precip_sel)
            wet_end=wet_end.where(precip_sel==0).where(precip_prev==1).fillna(0)
            ones_sum=ones_sum+precip_sel

            #want to subtract existing duration from places where wet spell ended
            duration_save=ones_sum.where(wet_end==1).fillna(0)
            ones_sum=ones_sum-duration_save
            
            ivt_current=ivt_sel.where(precip_sel==1)
            ivt_max=np.fmax(ivt_max,ivt_current.values)
            ivt_max_save=np.where(wet_end.values==1,ivt_max,0)
            ivt_max=np.subtract(ivt_max,ivt_max_save)
            ivt_max=np.where(ivt_max>0,ivt_max,np.nan)
            
            iwv_current=iwv_sel.where(precip_sel==1)
            iwv_max=np.fmax(iwv_max,iwv_current.values)
            iwv_max_save=np.where(wet_end.values==1,iwv_max,0)
            iwv_max=np.subtract(iwv_max,iwv_max_save)
            iwv_max=np.where(iwv_max>0,iwv_max,np.nan)
            
            s_current=s_sel.where(precip_sel==1)
            s_max=np.fmax(s_max,s_current.values)
            s_max_save=np.where(wet_end.values==1,s_max,0)
            s_max=np.subtract(s_max,s_max_save)
            s_max=np.where(s_max>0,s_max,np.nan)
            
            mfc_current=mfc_sel.where(precip_sel==1)
            mfc_max=np.fmax(mfc_max,mfc_current.values)
            mfc_max_save=np.where(wet_end.values==1,mfc_max,0)
            mfc_max=np.subtract(mfc_max,mfc_max_save)
            mfc_max=np.where(mfc_max>0,mfc_max,np.nan)

            if 1==0:
                
                fig=plt.figure(figsize=(18,12))
                ax=plt.subplot(1,1,1)
                test1=ones_sum.where(ones_sum>0)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,precip_sel.where(precip_sel>0).values,levels=np.arange(0,11,1),cmap=plt.cm.Blues)
                plt.show()
                plt.close()
                
                test=duration_save.where(duration_save>0)
                test1=ones_sum.where(ones_sum>0)
                fig=plt.figure(figsize=(18,12))
                ax=plt.subplot(1,1,1)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,test1.values,levels=np.arange(0,11,1),cmap=plt.cm.Blues)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,test.values,levels=np.arange(0,11,1),cmap=plt.cm.Reds)
                plt.show()
                plt.close()
                
                test=ivt_max_save
                fig=plt.figure(figsize=(18,12))
                ax=plt.subplot(1,1,1)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,test,levels=np.arange(0,1500,100),extend='max',cmap=plt.cm.seismic)
                plt.show()
                plt.close()
                
                test=ivt_max
                fig=plt.figure(figsize=(18,12))
                ax=plt.subplot(1,1,1)
                cax=ax.contourf(ones_sum.lon.values,ones_sum.lat.values,test,levels=np.arange(0,1500,100),extend='max',cmap=plt.cm.seismic)
                plt.show()
                plt.close()

            wet_spells[j,:,:]=duration_save.where(duration_save>0).values
            ivt_max_array[j,:,:]=np.where(ivt_max_save>0,ivt_max_save,np.nan)
            iwv_max_array[j,:,:]=np.where(iwv_max_save>0,iwv_max_save,np.nan)
            s_max_array[j,:,:]=np.where(s_max_save>0,s_max_save,np.nan)
            mfc_max_array[j,:,:]=np.where(mfc_max_save>0,mfc_max_save,np.nan)

        precip_prev=precip_sel
            
    ds.close()
    precip.close()
    
    #Now calculate means (so don't have to store huge datasets)
    
    #wet_spells=wet_spells.where(wet_spells>=2.)
    ivt_max_array=ivt_max_array.where(wet_spells>=2.)
    iwv_max_array=iwv_max_array.where(wet_spells>=2.)
    s_max_array=s_max_array.where(wet_spells>=2.)
    mfc_max_array=mfc_max_array.where(wet_spells>=2.)
    
    freq_4=wet_spells.where(wet_spells>=4.)
    freq_4=freq_4/freq_4
    freq_4=freq_4.sum(dim='time',skipna=True)
    freq_4=freq_4.expand_dims('time',0)
    
    duration_mean=wet_spells.mean(dim='time',skipna=True)
    ivt_max_mean=ivt_max_array.mean(dim='time',skipna=True)
    iwv_max_mean=iwv_max_array.mean(dim='time',skipna=True)
    s_max_mean=s_max_array.mean(dim='time',skipna=True)
    mfc_max_mean=mfc_max_array.mean(dim='time',skipna=True)
    
    if i==0:
        duration_concat=duration_mean
        ivt_max_concat=ivt_max_mean
        iwv_max_concat=iwv_max_mean
        s_max_concat=s_max_mean
        mfc_max_concat=mfc_max_mean
        freq_4_concat=freq_4
    else:
        duration_concat=xr.concat([duration_concat,duration_mean],dim='time')
        ivt_max_concat=xr.concat([ivt_max_concat,ivt_max_mean],dim='time')
        iwv_max_concat=xr.concat([iwv_max_concat,iwv_max_mean],dim='time')
        s_max_concat=xr.concat([s_max_concat,s_max_mean],dim='time')
        mfc_max_concat=xr.concat([mfc_max_concat,mfc_max_mean],dim='time')
        freq_4_concat=xr.concat([freq_4_concat,freq_4],dim='time')

dk=xr.Dataset()
dk['duration']=(('time','lat','lon'),duration_concat.values)
dk['max_ivt']=(('time','lat','lon'),ivt_max_concat.values)
dk['max_iwv']=(('time','lat','lon'),iwv_max_concat.values)
dk['max_s']=(('time','lat','lon'),s_max_concat.values)
dk['max_mfc']=(('time','lat','lon'),mfc_max_concat.values)
dk['freq_4']=(('time','lat','lon'),freq_4_concat.values)
dk.coords['time']=duration_concat.time
dk.coords['lat']=duration_concat.lat.values
dk.coords['lon']=duration_concat.lon.values
dk['duration'].attrs["description"]='length of AR recorded on timestep it ends (day AFTER last AR timestep).  Continuous from year to year except start of 1980.'
dk['max_ivt'].attrs["description"]='maximum IVT during duration of AR conditions ***for conditions that persist at least 2 time steps (6-12 hours)***.'  
dk['max_iwv'].attrs["description"]='maximum IWV during duration of AR conditions ***for conditions that persist at least 2 time steps (6-12 hours)***.'  
dk['max_s'].attrs["description"]='maximum 850 hPa wind speed during duration of AR conditions ***for conditions that persist at least 2 time steps (6-12 hours)***.'  
dk['max_mfc'].attrs["description"]='maximum vertically integrated MFC during duration of AR conditions ***for conditions that persist at least 2 time steps (6-12 hours)***.'  
dk['freq_4'].attrs["description"]='Frequency of AR conditions persisting for at least 4 6-hourly time steps'

try:
    os.remove(dir2+'merra2.ar.duration_chars.model.poleward.nc')
except OSError:
    pass
dk.to_netcdf(dir2+'merra2.ar.duration_chars.model.poleward.nc',mode='w',format='NETCDF4')

In [ ]:
#Below - checking means and trends
yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'merra2.ar.durations.model.variable.'+str(year)+'.nc')
    duration=ds['duration']
    duration=duration.where(duration>0)
    duration_mean=duration.mean(dim='time',skipna=True)
    
    if i==0:
        duration_concat=duration_mean
    else:
        duration_concat=xr.concat([duration_concat,duration_mean],dim='time')   

In [ ]:
field=duration_concat.mean(dim='time',skipna=True)/4.

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=0,vmax=1.25,cmap=plt.cm.BrBG,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('days',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{duration}$',fontsize=64,pad=5)

plt.show()

In [ ]:
slopes_array=xr.zeros_like(duration_concat[0,:,:])
sigs_array=xr.zeros_like(duration_concat[0,:,:])
for j in range(duration_concat.lat.size):
    print(j)
    for k in range(duration_concat.lon.size):
        time_series=duration_concat[:,j,k].values.tolist()
        r=st.linregress(yrs,time_series)
        slopes_array[j,k]=r[0]
        sigs_array[j,k]=r[3]

In [ ]:
field=slopes_array/4.

fig=plt.figure(figsize=(24,11))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-0.015,vmax=0.015,cmap=plt.cm.seismic,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=47) 
cbar.set_label('days yr$^{-1}$',fontsize=55,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':47,'color':'k'}
g1.ylabel_style={'size':47,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{duration}$'+' '+r'$\bf{trend}$',fontsize=64,pad=5)

plt.show()